# Image classification

- **Goal**: Show a standard workflow for a image classification task.
- **Dataset**: [MNIST](https://huggingface.co/datasets/ylecun/mnist), A dataset of 28x28 grayscale images that need to be classified as digits.
- **Libraries**: [datasets](https://github.com/huggingface/datasets), [transformers](https://github.com/huggingface/transformers)
- **Components**: [ImageField](https://docs.argilla.io/latest/reference/argilla/settings/fields/#src.argilla.settings._field.ImageField), [LabelQuestion](https://docs.argilla.io/latest/reference/argilla/settings/questions/#src.argilla.settings._question.LabelQuestion), [Suggestion](https://docs.argilla.io/latest/reference/argilla/records/suggestions/)

## Getting started

### Deploy the Argilla server

If you already have deployed Argilla, you can skip this step. Otherwise, you can quickly deploy Argilla following [this guide](../getting_started/quickstart.md).

### Set up the environment

To complete this tutorial, you need to install the Argilla SDK and a few third-party libraries via `pip`.

In [ ]:
!pip install argilla

In [ ]:
!pip install "transformers[torch]==4.3,<5"

Let's make the required imports:

In [76]:
import argilla as rg

import re
import io
import base64
from IPython.display import display

import torch
import numpy as np
from datasets import load_metric
from PIL import Image
from datasets import load_dataset, Dataset
from transformers import pipeline, AutoImageProcessor, AutoModelForImageClassification, Trainer, TrainingArguments, pipeline

You also need to connect to the Argilla server using the `api_url` and `api_key`.

In [3]:
# Replace api_url with your url if using Docker
# Replace api_key if you configured a custom API key
# Uncomment the last line and set your HF_TOKEN if your space is private
client = rg.Argilla(
    # api_url="https://[your-owner-name]-[your_space_name].hf.space",
    # api_url=
    api_key="argilla.apikey",
    # headers={"Authorization": f"Bearer {HF_TOKEN}"}
)

## Configure and create the Argilla dataset

Now, we will need to configure the dataset. In the settings, we can specify the guidelines, fields, and questions. If needed, you can also add metadata and vectors. However, for our use case, we just need a text field and a label question.

!!! note
    Check this [how-to guide](../how_to_guides/dataset.md) to know more about configuring and creating a dataset.

In [12]:
labels = [str(x) for x in range(10)]

settings = rg.Settings(
    guidelines="The goal of this task is to classify a given image of a handwritten digit into one of 10 classes representing integer values from 0 to 9, inclusively.",
    fields=[
        rg.ImageField(
            name="image",
            title="An image of a handwritten digit.",
        ),
    ],
    questions=[
        rg.LabelQuestion(
            name="image_label",
            title="What digit do you see on the image?",
            labels=labels,
        )
    ]
)

Let's create the dataset with the name and the defined settings:

In [ ]:
dataset = rg.Dataset(
    name="image_classification_dataset",
    settings=settings,
)
dataset.create()

In [4]:
dataset = client.datasets("image_classification_dataset")

## Add records

Even if we have created the dataset, it still lacks the information to be annotated (you can check it in the UI). We will use the `ylecun/mnist` dataset from [the Hugging Face Hub](https://huggingface.co/datasets/ylecun/mnist). Specifically, we will use the `train` split and get `100` examples. 

!!! tip
    When working with Hugging Face dataset you can set `Image(decode=False)` so that we can get [public image URLs](https://huggingface.co/docs/datasets/en/image_load#local-files), however, this depends on the dataset.

In [ ]:
hf_dataset = load_dataset("ylecun/mnist", split="train[:1000]")
hf_dataset

Let's have a look at the first image in the dataset.

In [6]:
hf_dataset[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
 'label': 5}

As we can seem, the image is a 28x28 grayscale image. In order to use it in in Argilla, we need to convert them to a base64 string.

In [17]:
def pil_to_data_uri(batch):
    data_uri = []
    for image in batch["image"]:
        buffered = io.BytesIO()
        image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode()
        data_uri.append(f"data:image/png;base64,{img_str}")
    batch["image_data_uri"] = data_uri
    return batch

hf_dataset= hf_dataset.map(pil_to_data_uri, batched=True)
hf_dataset[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
 'label': 5,
 'image_data_uri': ''}

We will easily add them to the dataset using `log` and the mapping, where we indicate that the column `text` is the data that should be added to the field `review`. We are also adding an "id" column to the record, so we can easily backtrack the record to the external data source.

In [ ]:
hf_dataset = hf_dataset.add_column("id", range(len(hf_dataset)))
dataset.records.log(records=hf_dataset[:100], mapping={"image_data_uri": "image"})

### Add initial model suggestions

The next step is to add suggestions to the dataset. This will make things easier and faster for the annotation team. Suggestions will appear as preselected options, so annotators will only need to correct them. In our case, we will generate them using [a zero-shot CLIP model](https://huggingface.co/docs/transformers/en/tasks/zero_shot_image_classification). However, you can use a framework or technique of your choice.

We will start by loading the model using a `transformers` pipeline.

In [10]:
checkpoint = "openai/clip-vit-large-patch14"
detector = pipeline(model=checkpoint, task="zero-shot-image-classification", device="mps")

Now, let's try to make a model prediction and see if it makes sense.

In [33]:
predictions = detector(hf_dataset[1]["image"], candidate_labels=labels)
predictions, display(hf_dataset[1]["image"])

([{'score': 0.5236628651618958, 'label': '0'},
  {'score': 0.11496700346469879, 'label': '7'},
  {'score': 0.08030630648136139, 'label': '8'},
  {'score': 0.07141078263521194, 'label': '9'},
  {'score': 0.05868939310312271, 'label': '6'},
  {'score': 0.05507850646972656, 'label': '5'},
  {'score': 0.0341767854988575, 'label': '1'},
  {'score': 0.027202051132917404, 'label': '4'},
  {'score': 0.018533246591687202, 'label': '3'},
  {'score': 0.015973029658198357, 'label': '2'}],
 None)

It's time to make the predictions on the dataset! We will set a function that uses the zero-shot model. The model will infer the label based on the image. When working with large datasets, you can create a `batch_predict` method to speed up the process.

In [52]:
def predict(input, labels):
    prediction = detector(input, candidate_labels=labels)
    prediction = prediction[0]
    return {"image_label": prediction["label"], "image_label.score": prediction["score"]}

To update the records, we will need to retrieve them from the server and update them with the new suggestions. The `id` will always need to be provided as it is the records' identifier to update a record and avoid creating a new one.

In [ ]:
data = dataset.records.to_list(flatten=True)
updated_data = [
    {
        "id": sample["id"],
        **predict(sample["image"], labels)
    }
    for sample in data
]
dataset.records.log(records=updated_data)

Voilà! We have added the suggestions to the dataset, and they will appear in the UI marked with a ✨. 

## Evaluate with Argilla

Now, we can start the annotation process. Just open the dataset in the Argilla UI and start annotating the records. If the suggestions are correct, you can just click on `Submit`. Otherwise, you can select the correct label.

!!! note
    Check this [how-to guide](../how_to_guides/annotate.md) to know more about annotating in the UI.

## Train your model

After the annotation, we will have a robust dataset to train the main model. In our case, we will fine-tune using transformers and the . However, you can select the one that best fits your requirements. So, let's start by retrieving the annotated records.

!!! note
    Check this [how-to guide](../how_to_guides/query.md) to know more about filtering and querying in Argilla. Also, you can check the Hugging Face docs on [fine-tuning an image classification model](https://huggingface.co/docs/transformers/en/tasks/image_classification).

In [16]:
dataset = client.datasets("image_classification_dataset")

In [ ]:
status_filter = rg.Query(filter=rg.Filter(("response.status", "==", "submitted")))

submitted = dataset.records(status_filter).to_list(flatten=True)

We then need to convert our base64 images to a format that the model can understand so we will convert them to PIL images again.

In [55]:
def base64_to_pil(base64_string):
    image_data = re.sub('^data:image/.+;base64,', '', base64_string)
    image = Image.open(io.BytesIO(base64.b64decode(image_data)))
    return image

Now, let's apply that to the whole dataset.

In [ ]:
submitted_pil_image = [
    {
        "id": sample["id"],
        "image": base64_to_pil(sample["image"]),
        "label": sample["image_label.responses"][0],
    }
    for sample in submitted
]
submitted_pil_image[0]

We now need to ensure our images are forwarded with the correct dimensions. Because the original MNIST dataset is greyscale and the VIT model expects RGB, we need to add a channel dimension to the images. We will do this by stacking the images along the channel axis.

In [ ]:
def greyscale_to_rgb(img) -> Image:
    return Image.merge('RGB', (img, img, img))

submitted_pil_image_rgb = [
    {
        "image": greyscale_to_rgb(sample["image"]),
        "label": sample["label"],
    }
    for sample in submitted_pil_image
]
submitted_pil_image_rgb[0]

Next, we will load the `ImageProcessor` for fine-tuning the model. This processor will handle the image resizing and normalization in order to be compatible with the model we intend to use.

In [ ]:
checkpoint = "google/vit-base-patch16-224-in21k"
processor = AutoImageProcessor.from_pretrained(checkpoint)

submitted_pil_image_rgb_processed = [
    {
        "pixel_values": processor(sample["image"], return_tensors='pt')["pixel_values"],
        "label": sample["label"],
    }
    for sample in submitted_pil_image_rgb
]
submitted_pil_image_rgb_processed[0]

We can now convert the images to a Hugging Face datasets Dataset that is ready for fine-tuning.

In [ ]:
prepared_ds = Dataset.from_list(submitted_pil_image_rgb_processed)
prepared_ds = prepared_ds.train_test_split(test_size=0.2)
prepared_ds

We then need to define our data collator, which will ensure the data is unpacked and stacked correctly for the model. We wi

In [ ]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([torch.tensor(x['pixel_values'][0]) for x in batch]),
        'labels': torch.tensor([int(x['label']) for x in batch])
    }

Next, we can define our training metrics. We will use the accuracy metric to evaluate the model's performance.

In [ ]:
metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

We then load our model and configure the labels that we will use for training.

In [ ]:
model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label={int(i): int(c) for i, c in enumerate(labels)},
    label2id={int(c): int(i) for i, c in enumerate(labels)}
)
model.config

Finally, we define the training arguments and start the training process.

In [ ]:
training_args = TrainingArguments(
  output_dir="./image-classifier",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=1,
  fp16=False, # True if you have a GPU with mixed precision support
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=True,
  push_to_hub=False,
  load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["test"],
    tokenizer=processor,
)

train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

As the training data had a better-quality, we can expect a better model. So, we can update the remainder of our original dataset with the new model's suggestions.

In [ ]:
pipe = pipeline("image-classification", model=model, image_processor=processor)

def run_inference(batch):
    predictions = pipe(batch["image"])
    batch["image_label"] = [prediction[0]["label"] for prediction in predictions]
    batch["image_label.score"] = [prediction[0]["score"] for prediction in predictions]
    return batch

hf_dataset = hf_dataset.map(run_inference, batched=True)
dataset.records.log(records=hf_dataset[:100], mapping={"image_data_uri": "image"})

## Conclusions

In this tutorial, we present an end-to-end example of a image classification task. This serves as the base, but it can be performed iteratively and seamlessly integrated into your workflow to ensure high-quality curation of your data and improved results.

We started by configuring the dataset, adding records, as an example, to add suggestions. After the annotation process, we trained a new model with the annotated data and updated the remaining records with the new suggestions.